Exaple of how to programmatically define a Lambda function and invoke it:

In [88]:
import binascii
import os
import requests
import boto3
import time
import random
import json
import inspect
import io
import zipfile

from botocore.exceptions import ClientError

This will be our Lambda function:

In [89]:
def handler(event, context):
    message = f'Hello {event["first_name"]} {event["last_name"]}!'
    return {'message': message, 'event': event}

handler({"first_name":"Fred", "last_name":"Krueger"}, None)

{'message': 'Hello Fred Krueger!',
 'event': {'first_name': 'Fred', 'last_name': 'Krueger'}}

#### We'll tag everything as follows:

In [90]:
tags=[
    {
        'Key': 'Environment',
        'Value': 'Production'
    }
]

#### First, define a role for executing our lambda function:

In [91]:
role={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

roleName="MyLambdaRole"

iam = boto3.client('iam')

try:
    r = iam.create_role(RoleName=roleName, 
                Path="/",
                Description="Example role for testing",
                AssumeRolePolicyDocument=json.dumps(role),
                Tags=tags)
    print("Role added.")
except ClientError as e:
    if e.response['Error']['Code'] != 'EntityAlreadyExists':
        raise
    print("Role already exists.")

lambdaExecRole = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
r = iam.attach_role_policy(RoleName=roleName, 
                       PolicyArn=lambdaExecRole)
print("Lambda policy attached.")

roleArn = iam.get_role(RoleName=roleName)['Role']['Arn']
print("Role's ARN is "+roleArn)

Role added.
Lambda policy attached.
Role's ARN is arn:aws:iam::549935274340:role/MyLambdaRole


#### Zip our function:

In [92]:
lambda_zip = io.BytesIO()

fileInfo = zipfile.ZipInfo("hello.py")
fileInfo.external_attr = 0o777 << 16

with zipfile.ZipFile(lambda_zip, mode="w",compression=zipfile.ZIP_DEFLATED) as zf:
    zf.writestr(fileInfo, inspect.getsource(handler))

print(str(len(lambda_zip.getvalue()))+" bytes in ZIP file")

258 bytes in ZIP file


#### Upload it as a new Lambda function:

In [95]:
awslambda=boto3.client('lambda')

try:
    awslambda.create_function(
        FunctionName="HelloWorld",
        Runtime='python3.6',
        Role=roleArn,
        Handler='hello.handler',
        Code={'ZipFile':lambda_zip.getvalue()})
    print("Lambda created.")
except ClientError as e:
    if e.response['Error']['Code'] != 'ResourceConflictException':
        raise
    print("Lambda already exists.")

Lambda created.


#### And now invoke the lambda:

In [80]:
event = {'first_name': 'Jennifer', 'last_name': 'Deere'}

response = awslambda.invoke(
        FunctionName='HelloWorld',
        InvocationType='RequestResponse',
        LogType='Tail',
        Payload=json.dumps(event))

print(response['Payload'].read())

b'{"message": "Hello Jennifer Deere!", "event": {"first_name": "Jennifer", "last_name": "Deere"}}'


#### Check the logs:

In [81]:
from datetime import datetime
log = boto3.client('logs')
for event in log.filter_log_events(logGroupName=f'/aws/lambda/HelloWorld')['events']:
    timestamp = int(event['timestamp'])/1000
    date = str(datetime.fromtimestamp(timestamp))[:23]
    message = event['message'].strip()
    print(date, message)

2019-12-19 10:30:03.885 START RequestId: 4dd86d71-c440-4479-9f5f-5d6bb1c6f565 Version: $LATEST
2019-12-19 10:30:03.885 module initialization error: [Errno 13] Permission denied: '/var/task/hello.py'
2019-12-19 10:30:04.117 END RequestId: 4dd86d71-c440-4479-9f5f-5d6bb1c6f565
2019-12-19 10:30:04.117 REPORT RequestId: 4dd86d71-c440-4479-9f5f-5d6bb1c6f565	Duration: 232.08 ms	Billed Duration: 300 ms	Memory Size: 128 MB	Max Memory Used: 50 MB	Init Duration: 1.05 ms
2019-12-19 10:30:04.117 module initialization error
[Errno 13] Permission denied: '/var/task/hello.py'
2019-12-19 10:30:23.907 START RequestId: 8bb00ffa-4ef7-4b1e-9a8b-9407a69c5fb5 Version: $LATEST
2019-12-19 10:30:23.909 module initialization error: [Errno 13] Permission denied: '/var/task/hello.py'
2019-12-19 10:30:24.187 END RequestId: 8bb00ffa-4ef7-4b1e-9a8b-9407a69c5fb5
2019-12-19 10:30:24.187 REPORT RequestId: 8bb00ffa-4ef7-4b1e-9a8b-9407a69c5fb5	Duration: 276.79 ms	Billed Duration: 300 ms	Memory Size: 128 MB	Max Memory Used

#### Housekeeping: Delete everything we've explicitely or implicitely created:

In [96]:
try:
    awslambda.delete_function(FunctionName='HelloWorld')
except ClientError as e:
    if e.response['Error']['Code'] != 'ResourceNotFoundException':
        raise

try:
    iam.detach_role_policy(RoleName=roleName, PolicyArn=lambdaExecRole)
    iam.delete_role(RoleName=roleName)
except ClientError as e:
    if e.response['Error']['Code'] != 'NoSuchEntity':
        raise
        
try:
    log.delete_log_group(logGroupName=f'/aws/lambda/HelloWorld')
except ClientError as e:
    if e.response['Error']['Code'] != 'ResourceNotFoundException':
        raise